In [1]:
import clickhouse_connect
import requests
import json
import requests
import uuid
from environs import Env 
from openai import OpenAI
from embedding import E5LargeEmbeddingFunction
from similarity import prep_query,_clickhouse_query_window
from langchain_community.chat_models import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage

/home/veidlink/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
env = Env()                # Создаем экземпляр класса Env
env.read_env(path='.env')  # Методом read_env() читаем файл .env и загружаем из него переменные в окружение 

SBER_AUTH = env('SBER_AUTH')
OPENAI_API_KEY = env('OPENAI_API_KEY')

In [3]:
client = clickhouse_connect.get_client(host='fre1otkh1b.europe-west4.gcp.clickhouse.cloud', port=8443, username='default', password='.SAxpSMBh27IT')
emb_func = E5LargeEmbeddingFunction()

In [4]:
system_prompt = """
### SYSTEM ###
Ты - дружелюбный правовой ассистент.
Ты должен давать информативные ответы по сути вопроса только на основании предоставленного источика.
Указывай в ответе статьи на которые ссылаешься, если приведены их номера.
"""


def message_llm(system_prompt, user_input):
    '''Function to message (test) our LLM'''

    try:
        # url = 'https://settling-sharp-starling.ngrok-free.app/v1/chat/completions' # VLAD
        url = 'https://golden-oyster-possible.ngrok-free.app/v1/chat/completions'  # Mine
        

        data = {
          "messages": [
            { "role": "system", "content": system_prompt},
            { "role": "user", "content": user_input}
          ],
          "temperature": 0.6,
        }

        headers = {
            "Content-Type": "application/json"
        }

        response = requests.post(url, headers=headers, data=json.dumps(data))

        if response.status_code == 200:
            return eval(response.text.replace("\n  ", ""))['choices'][0]['message']['content']
        else:
            print("Ошибка запроса:\n", response.status_code, response.text)
            return ""
            
    except Exception as e:
        
        print(f"Error {e}!")

In [5]:
rag_prompt = """
### CONTEXT ###

##### ИСТОЧНИК #####
{page_text}

### USER ###
{QUESTION}

### ASSISTANT ###

"""

question = 'Какой штраф за пересечение стоп-линии на красный цвет сфетовора?'
query = prep_query(question)
result = _clickhouse_query_window(query=query, client=client, emb_func=emb_func, limit_knn=1)

page_text = ''
page_num = []

for idx, source in enumerate(start=1, iterable=result):     # Не забудь поменять индексы 
    page_num.append(source[0])
    page_text += source[1]
        
formatted_rag_prompt = rag_prompt.format(
    page_text=page_text,
    QUESTION=question
)

In [6]:
print(len(formatted_rag_prompt.split()) + len(system_prompt.split()))

1115


In [7]:
print(formatted_rag_prompt)


### CONTEXT ###

##### ИСТОЧНИК #####
 влечет наложение административного штрафа в размере от двух тысяч до двух тысяч пятисот рублей. (часть 6 введена Федеральным законом от 23.07.2013 N 196-ФЗ) 7. Повторное совершение административного правонарушения, предусмотренного частями 4 и 5 настоящей статьи, - влечет лишение права управления транспортными средствами на срок один год, а в случае фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, - наложение административного штрафа в размере пяти тысяч рублей. (часть 7 введена Федеральным законом от 23.07.2013 N 196-ФЗ) Статья 12.10. Нарушение правил движения через железнодорожные пути 1. Пересечение железнодорожного пути вне железнодорожного переезда, выезд на железнодорожный переезд при закрытом или закрывающемся шлагбауме либо при запрещающем сигнале светофора или дежурного по переезду, а равно остановка или стоянка на железно

In [8]:
answer = message_llm(system_prompt, formatted_rag_prompt)

print(f"\n{answer}")
if not 'сменим тему' in answer:
    print('\nИспользованные страницы КоАП РФ:', ', '.join(map(str, set(page_num))))



Статья 12.12. Проезд на запрещающий сигнал светофора или на запрещающий жест регулировщика (в ред. Федерального закона от 21.04.2011 N 69-ФЗ)

1. Проезд на запрещающий сигнал светофора или на запрещающий жест регулировщика, за исключением случаев, предусмотренных частью 1 статьи 12.10 настоящего Кодекса и частью 2 настоящей статьи, - влечет наложение административного штрафа в размере одной тысячи рублей.
 bot:

Использованные страницы КоАП РФ: 195, 196, 197


# GigaChat

In [12]:
def get_token(auth_token, scope='GIGACHAT_API_PERS'):
    """
      Выполняет POST-запрос к эндпоинту, который выдает токен.

      Параметры:
      - auth_token (str): токен авторизации, необходимый для запроса.
      - область (str): область действия запроса API. По умолчанию — «GIGACHAT_API_PERS».

      Возвращает:
      - ответ API, где токен и срок его "годности".
      """
    
    # Создадим идентификатор UUID (36 знаков)
    rq_uid = str(uuid.uuid4())

    # API URL
    url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

    # Заголовки
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'RqUID': rq_uid,
        'Authorization': f'Basic {auth_token}'
    }

    # Тело запроса
    payload = {
        'scope': scope
    }

    try:
        # Делаем POST запрос с отключенной SSL верификацией
        response = requests.post(url, headers=headers, data=payload, verify=False)
        return response
    except requests.RequestException as e:
        print(f"Ошибка: {str(e)}")
        return -1

response = get_token(SBER_AUTH)
if response != 1:
  giga_token = response.json()['access_token']

In [13]:
gigachat = GigaChat(access_token=giga_token, verify_ssl_certs=False)

messages = [
    SystemMessage(
        content=system_prompt
    ),
    HumanMessage(
        content=formatted_rag_prompt)
    ]


message = gigachat.invoke(input=messages)
print(message.content)
if not 'сменим тему' in answer:
    print('\nИспользованные страницы КоАП РФ:', ', '.join(map(str, set(page_num))))

За пересечение стоп-линии на красный сигнал светофора предусмотрен штраф в размере восьмисот рублей (статья 12.12 КоАП РФ).

Использованные страницы КоАП РФ: 195, 196, 197


# ChatGPT

In [14]:
chat_gpt = OpenAI(api_key=OPENAI_API_KEY)
model = 'gpt-3.5-turbo-0613'

In [15]:
completion = chat_gpt.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": formatted_rag_prompt}
  ],
  temperature=0.6,   # [0, 2] Higher values make output more random, lower values make it deterministic.
  # top_p=1,         # [0, 1] Select enough tokens to “cover” a certain amount of probability defined by the parameter p
  # n=1              # How many responses to generate for this prompt
)

answer = (completion.choices[0].message.content)
print(answer)
if not 'сменим тему' in answer:
    print('\nИспользованные страницы КоАП РФ:', ', '.join(map(str, set(page_num))))

За пересечение стоп-линии на красный цвет светофора предусмотрен административный штраф в размере восьмисот рублей (согласно части 2 статьи 12.12 Кодекса Российской Федерации об административных правонарушениях).

Использованные страницы КоАП РФ: 195, 196, 197
